In [1]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import glob
import PIL
from PIL import Image, ImageEnhance, ImageFilter
from tqdm import tqdm
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
import logging
from pathlib import Path
import shutil
import time
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Model, Sequential
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D, Input, Reshape
from keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf

from sklearn.model_selection import train_test_split
from warnings import filterwarnings

filterwarnings('ignore')
plt.rcParams["axes.grid"] = False
np.random.seed(101)

# Constants
IMAGE_SIZE = (224, 224)
SUPPORTED_FORMATS = ['.jpg', '.jpeg', '.png']

In [4]:
class ImagePreprocessor:
    """Skin Cancer Image preprocessing pipeline"""
    
    @staticmethod
    def hair_remove(image):
        """Remove hair from skin images"""
        try:
            grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17, 17))
            blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
            _, threshold = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
            final_image = cv2.inpaint(image, threshold, 1, cv2.INPAINT_TELEA)
            return final_image
        except Exception as e:
            print(f"Error in hair removal: {str(e)}")
            return image

    @staticmethod
    def sharpen_image(image):
        """Sharpen image using unsharp masking"""
        gaussian = cv2.GaussianBlur(image, (0, 0), 2.0)
        return cv2.addWeighted(image, 1.5, gaussian, -0.5, 0)

In [5]:
def preprocess_image(image, target_size=(224, 224)):
    """Apply all preprocessing steps to an image"""
    preprocessor = ImagePreprocessor()
    
    image = preprocessor.hair_remove(image)
    image = preprocessor.sharpen_image(image)
    image = cv2.resize(image, target_size, interpolation=cv2.INTER_NEAREST)
    
    return image

In [6]:
def iou(y_true, y_pred, smooth = 100):
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.square(y_true), axis = -1) + K.sum(K.square(y_pred), axis=-1)
    union = sum_ - intersection
    jac = (intersection + smooth) / (union + smooth)
    return jac

def dice_coef(y_true, y_pred, smooth = 100):
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def precision(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
   
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [7]:
# Encoding layer

img_input = Input(shape= (224, 224, 3))
x = Conv2D(64, (3, 3), padding='same', name='conv1',strides= (1,1))(img_input)
x = BatchNormalization(name='bn1')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), padding='same', name='conv2')(x)
x = BatchNormalization(name='bn2')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(128, (3, 3), padding='same', name='conv3')(x)
x = BatchNormalization(name='bn3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), padding='same', name='conv4')(x)
x = BatchNormalization(name='bn4')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(256, (3, 3), padding='same', name='conv5')(x)
x = BatchNormalization(name='bn5')(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same', name='conv6')(x)
x = BatchNormalization(name='bn6')(x)
x = Activation('relu')(x)
x = Conv2D(256, (3, 3), padding='same', name='conv7')(x)
x = BatchNormalization(name='bn7')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(512, (3, 3), padding='same', name='conv8')(x)
x = BatchNormalization(name='bn8')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv9')(x)
x = BatchNormalization(name='bn9')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv10')(x)
x = BatchNormalization(name='bn10')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Conv2D(512, (3, 3), padding='same', name='conv11')(x)
x = BatchNormalization(name='bn11')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv12')(x)
x = BatchNormalization(name='bn12')(x)
x = Activation('relu')(x)
x = Conv2D(512, (3, 3), padding='same', name='conv13')(x)
x = BatchNormalization(name='bn13')(x)
x = Activation('relu')(x)
x = MaxPooling2D()(x)

x = Dense(1024, activation = 'relu', name='fc1')(x)
x = Dense(1024, activation = 'relu', name='fc2')(x)

# Decoding Layer 

x = UpSampling2D()(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv1')(x)
x = BatchNormalization(name='bn14')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv2')(x)
x = BatchNormalization(name='bn15')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv3')(x)
x = BatchNormalization(name='bn16')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv4')(x)
x = BatchNormalization(name='bn17')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(512, (3, 3), padding='same', name='deconv5')(x)
x = BatchNormalization(name='bn18')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv6')(x)
x = BatchNormalization(name='bn19')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv7')(x)
x = BatchNormalization(name='bn20')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(256, (3, 3), padding='same', name='deconv8')(x)
x = BatchNormalization(name='bn21')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv9')(x)
x = BatchNormalization(name='bn22')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(128, (3, 3), padding='same', name='deconv10')(x)
x = BatchNormalization(name='bn23')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv11')(x)
x = BatchNormalization(name='bn24')(x)
x = Activation('relu')(x)

x = UpSampling2D()(x)
x = Conv2DTranspose(64, (3, 3), padding='same', name='deconv12')(x)
x = BatchNormalization(name='bn25')(x)
x = Activation('relu')(x)
x = Conv2DTranspose(1, (3, 3), padding='same', name='deconv13')(x)
x = BatchNormalization(name='bn26')(x)
x = Activation('sigmoid')(x)
pred = Reshape((224,224))(x)


In [8]:
segnet_model = Model(inputs=img_input, outputs=pred)
segnet_model.compile(optimizer= Adam(learning_rate = 0.003), loss= ['binary_crossentropy']
              , metrics=[iou, dice_coef, precision, recall, accuracy])

In [9]:
segnet_model.load_weights('/kaggle/input/segnet-segmentation-weights/segnet_120_epoch.h5')

In [10]:
def enhance(img):
    sub = (segnet_model.predict(img.reshape(1,224,224,3))).flatten()

    for i in range(len(sub)):
        if sub[i] > 0.5:
            sub[i] = 1
        else:
            sub[i] = 0
    return sub

In [11]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import cv2
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def process_and_organize_dataset(source_path, destination_path, batch_size=100):
    """Process images in batches and organize them into the new structure"""
    source_path = Path(source_path)
    destination_path = Path(destination_path)
    splits = ['train_directory', 'test_directory', 'validation_directory']

    def process_batch(image_batch, filenames):
        # Predict masks for entire batch at once
        masks = segnet_model.predict(np.array(image_batch), batch_size=len(image_batch))
        masks = (masks > 0.5).astype(np.uint8)
        
        # Process each image and its mask
        for idx, (img, mask, filename) in enumerate(zip(image_batch, masks, filenames)):
            # Reshape mask back to 2D
            mask = mask.reshape(224, 224)
            
            # Save mask
            mask_save = (mask * 255).astype(np.uint8)
            mask_path = current_mask_path / f"{filename}.jpg"
            cv2.imwrite(str(mask_path), mask_save)
            
            # Create and save segmented image
            segmented = img.copy()
            segmented[mask == 0] = 0
            segmented_path = current_segment_path / f"{filename}.jpg"
            cv2.imwrite(str(segmented_path), cv2.cvtColor(segmented, cv2.COLOR_RGB2BGR))

    for split in splits:
        split_path = source_path / split
        dest_split = split
        
        for category in ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']:
            category_path = split_path / category
            if not category_path.exists():
                continue
                
            print(f"Processing {split}/{category}...")
            
            # Create output directories
            current_mask_path = destination_path / 'ground_truth' / dest_split / category
            current_segment_path = destination_path / 'segnet_segmented' / dest_split / category
            current_mask_path.mkdir(parents=True, exist_ok=True)
            current_segment_path.mkdir(parents=True, exist_ok=True)
            
            # Collect all images
            image_files = [f for ext in SUPPORTED_FORMATS for f in category_path.glob(f'*{ext}')]
            
            # Process in batches
            batch_images = []
            batch_filenames = []
            
            for img_file in tqdm(image_files):
                img = cv2.imread(str(img_file))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                processed_img = preprocess_image(img.copy(), target_size=(224, 224))
                
                batch_images.append(processed_img)
                batch_filenames.append(img_file.stem)
                
                # Process batch when it reaches batch_size or at the end
                if len(batch_images) == batch_size or img_file == image_files[-1]:
                    process_batch(batch_images, batch_filenames)
                    batch_images = []
                    batch_filenames = []

In [13]:
base_directory = '/kaggle/working/segnet_segmented_images'
os.mkdir(base_directory)

subfolders = ['ground_truth', 'segnet_segmented']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    path = os.path.join(base_directory, subf)
    os.mkdir(path)
    for dirc in directory:
        path = os.path.join(base_directory, subf, dirc)
        os.mkdir(path)
        for cls in classes:
            path = os.path.join(base_directory, subf, dirc, cls)
            os.mkdir(path)

In [14]:
source_path = "/kaggle/input/multiclassskincancer"
destination_path = "/kaggle/working/segnet_segmented_images"
process_and_organize_dataset(source_path, destination_path)

Processing train_directory/nv...


  2%|▏         | 98/5115 [00:19<04:45, 17.60it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step


  4%|▍         | 197/5115 [00:41<03:32, 23.12it/s]  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


...

100%|██████████| 10/10 [00:00<00:00, 16.23it/s]


Processing validation_directory/df...


 67%|██████▋   | 4/6 [00:00<00:00, 15.01it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


100%|██████████| 6/6 [00:03<00:00,  1.74it/s]


In [15]:
base_dir = '/kaggle/working/segnet_segmented_images'
subfolders = ['ground_truth', 'segnet_segmented']
directory = ['train_directory', 'test_directory', 'validation_directory']
classes = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for subf in subfolders:
    print(subf)
    for dirc in directory:
        print(dirc)
        for cls in classes:
            path = os.path.join(base_dir, subf, dirc, cls)
            print(f"{cls}    : ", len(os.listdir(path)))

ground_truth
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
segnet_segmented
train_directory
nv    :  5115
mel    :  5950
bkl    :  5990
bcc    :  5462
akiec    :  5510
vasc    :  4810
df    :  4090
test_directory
nv    :  883
mel    :  46
bkl    :  88
bcc    :  35
akiec    :  30
vasc    :  13
df    :  8
validation_directory
nv    :  707
mel    :  37
bkl    :  71
bcc    :  28
akiec    :  24
vasc    :  10
df    :  6
